# 创建一个类

github: https://github.com/fluentpython/example-code/tree/master/09-pythonic-obj



In [38]:
# vector 2d class
import math
from array import array

class Vector2d:
    typecode = 'd' #typecode是类属性，在Vector2d实例和字节序列之间转换时使用。

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return "{}({!r},{!r})".format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
                bytes(array(self.typecode, self)))

    
    def angle(self):
        return math.atan2(self.x, self.y)

    
    # 备选构造方法 （与上面的方法不同，传入的参数不同）
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        mev = memoryview(octets[1:]).cast(typecode)
        return cls(*mev)

    # 格式化显示
    def __format__(self, fmt_spec):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer = '<{},{}>'
        else:
            coords = (i for i in self)
            outer = '({},{})'
        components = (format(c, fmt_spec) for c in coords)
        return outer.format(*components)



In [39]:
# 构造函数
v1 = Vector2d(1,2)
print("__init__():Vector2d(1,2)")

# 可拆包
vx, vy = v1
print("__iter__():{}, {}".format(vx, vy))

# 支持打印。如果没有定义str会使用repr函数代替
print("__str__():{}".format(v1))

# eval with format
v2 = eval(repr(v1))
print("__repr__():eval(repr(v1))-->{}".format(v2))

# equal
print("__eq__():{}".format(v1 == v2))

# abs
print("__abs__(): abs(v1)-->{}".format(abs(v1)))

# bool
print("__bool__(): bool(v1)-->{}".format(bool(v1)))

# bytes
print("__bytes__(): bytes(v1)-->{}".format(bytes(v1)))

__init__():Vector2d(1,2)
__iter__():1.0, 2.0
__str__():(1.0,2.0)
__repr__():eval(repr(v1))-->(1.0,2.0)
__eq__():True
__abs__(): abs(v1)-->2.23606797749979
__bool__(): bool(v1)-->True
__bytes__(): bytes(v1)-->b'd\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00@'


# classmethod vs staticmethod

In [40]:
# classmethod是操作类的方法，而不是操作实例的方法。
Vector2d.frombytes(bytes(v1))

Vector2d(1.0,2.0)

In [41]:
# staticmethod 是一个静态函数，即使写在了类中也有全局效果，类似与直接写在模块中的类。

def common_static_method(*args):
     return args

class Demo:
     @staticmethod
     def static_method(*args):
          return args
     
     @classmethod
     def class_method(*args):
          return args
           


In [42]:
# 作为类的方法，返回的第一个参数永远是类本身
Demo.class_method()

(__main__.Demo,)

In [43]:
Demo.class_method("test")

(__main__.Demo, 'test')

In [44]:
# 而作为静态方法，返回的函数与直接在模块中创建的函数返回的内容是一样的
Demo.static_method()

()

In [45]:
Demo.static_method("test")

('test',)

In [46]:
common_static_method()

()

In [47]:
common_static_method("test")

('test',)

# 格式化表示

In [48]:
print("{v:0.1f}".format(v = v1))

(1.0,2.0)


In [49]:
format(v1, '0.3f')

'(1.000,2.000)'

In [50]:
# 使用自己定义的格式打印
format(v1, '0.1fp')

'<2.2,0.5>'

# 可散列

In [69]:
# 位运算符异或计算实例：
a = 443.016
b = 444.017
print(hash(a))
print(hash(b))
hash(a)^hash(b)

36893488147464635
39199331156623804


2341938670338055

# 声明只读变量与可散列

其实只要类中实现\_\_Hash\_\_()和\_\_eq\_\_()两个函数，就能实现散列。但是因为散列要求数据只读，以此我们需要在类中使用特性将变量声明成只读属性。

In [80]:
class pVector2d:
    def __init__(self, x, y):
        self.__x = x
        self.__y = y

    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __repr__(self):
        class_name = type(self).__name__
        return "{}({},{})".format(class_name, self.x, self.y)

    # 只有只读属性，才能散列
    def __hash__(self):
        return hash(self.x)^hash(self.y)

In [79]:
pv1 = pVector2d(2,3)
# 只读属性
pv1.x = 1

AttributeError: can't set attribute

In [81]:
# v1 不可散列
v1

Vector2d(1.0,2.0)

In [82]:
# pv1 可散列
pv1

pVector2d(2,3)

In [85]:
# v1 不可散列：unhashable
set([v1,pv1])

TypeError: unhashable type: 'Vector2d'

In [86]:
hash(v1)

TypeError: unhashable type: 'Vector2d'

In [87]:
hash(pv1)

8775947394067

# 整体实现

- 实现分量表示
- 实现拆包
- 实现格式化
- 实现repr返回构建源码（eval可以创建实例）
- 实现相等
- 实现返回布尔值
- 实现打印
- 实现操作类方法
- 实现返回绝对值
- 实现返回Bytes值
- 实现散列
- 实现只读
- 实现float转换
- 实现int转换
- 实现complex


In [4]:
import math
from array import array

class myVector2d:

    typecode = 'd'

    def __init__(self, x, y):
        self.__x = x
        self.__y = y

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return ( i for i in (self.x, self.y))

    def __abs__(self):
        #return math.sqrt(self.x * self.x + self.y * self.y)
        return math.hypot(self.x, self.y) # hypot求斜边
    
    def __repr__(self):
        return "myVector2d({},{})".format(self.x,self.y)

    def __str__(self):
        return str(tuple(self))

    def __hash__(self):
        return hash(self.x)^hash(self.y)

    def __eq__(self, other):
        return hash(self) == hash(other)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __bytes__(self):
        # 二进制表示形式
        return (bytes([ord(self.typecode)]) + 
                bytes(array(self.typecode, [self.x, self.y])))

    def angle(self):
        return math.atan2(self.y, self.x)    

    def __format__(self, fmt_spec):
        if fmt_spec.endswith("p") :
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            out = "myVector2d:<{},{}>"
        else:
            coords = (i for i in self)
            out = "myVector2d:({},{})"
        componets = (format(c, fmt_spec) for c in coords)
        return out.format(*componets)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        mev = memoryview(octets[1:]).cast(typecode)
        return cls(*mev)

    def __float__(self):
        return float(abs(self))

    def __int__(self):
        return int(abs(self))

    def __complex__(self):
        return complex(self.x, self.y)

In [2]:
mv = myVector2d(2,5)

In [3]:
# 验证repr
mv

myVector2d(2,5)

In [4]:
# 验证拆包和分量
a,b = mv
print("a:{}\nb:{}\nmv.x:{}\nmv.y:{}\n".format(a,b,mv.x,mv.y))

a:2
b:5
mv.x:2
mv.y:5



In [5]:
# 验证str
print(mv)

(2, 5)


In [6]:
# 验证abs
abs(mv)

5.385164807134505

In [7]:
# 验证相等
mv2 = myVector2d(2,3)

mv == mv2

False

In [8]:
mv3 = mv
mv == mv3

True

In [9]:
# 验证可散列
set([mv,mv2,mv3])

{myVector2d(2,3), myVector2d(2,5)}

In [10]:
# 验证布尔值
bool(mv)

True

In [11]:
# 验证格式化
"{:0.3fp}".format(mv)

'myVector2d:<5.385,1.190>'

In [12]:
# 验证格式化2
"{:0.3f}".format(mv)

'myVector2d:(2.000,5.000)'

In [13]:
# 验证二进制转换
bytes(mv2)

b'd\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x08@'

In [14]:
# 验证类方法
octets = bytes(mv2)
result = myVector2d.frombytes(octets)
result

myVector2d(2.0,3.0)

In [15]:
# 验证float和int
float(mv)

5.385164807134505

In [16]:
int(mv)

5

In [17]:
# 验证complex

complex(mv)

(2+5j)

# 使用 __slots__ 类属性节省空间

In [18]:
# 实例默认使用字典记录其属性
print(mv.__dict__)

# 如果有数百万个属性不多的实例，效率就会大大降低。

{'_myVector2d__x': 2, '_myVector2d__y': 5}


## 通过 \_\_slots\_\_ 类属性，能节省大量内存,方法是让解释器在元组中存储实例属性，而不用字典。

In [1]:
import math
from array import array

class myVector2d_v02:

    """
    在类中定义 __slots__ 属性的目的是告诉解释器:“这个类中的所有 实例属性都在这儿了!”这样，Python 会在各个实例中使用类似元组的 结构存储实例变量，从而避免使用消耗内存的 __dict__ 属性。如果 有数百万个实例同时活动，这样做能节省大量内存。
    """
    __slots__ = ('__x','__y')

    typecode = 'd'

    def __init__(self, x, y):
        self.__x = x
        self.__y = y

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return ( i for i in (self.x, self.y))

    def __abs__(self):
        #return math.sqrt(self.x * self.x + self.y * self.y)
        return math.hypot(self.x, self.y) # hypot求斜边
    
    def __repr__(self):
        return "myVector2d({},{})".format(self.x,self.y)

    def __str__(self):
        return str(tuple(self))

    def __hash__(self):
        return hash(self.x)^hash(self.y)

    def __eq__(self, other):
        return hash(self) == hash(other)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __bytes__(self):
        # 二进制表示形式
        return (bytes([ord(self.typecode)]) + 
                bytes(array(self.typecode, [self.x, self.y])))

    def angle(self):
        return math.atan2(self.y, self.x)    

    def __format__(self, fmt_spec):
        if fmt_spec.endswith("p") :
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            out = "myVector2d:<{},{}>"
        else:
            coords = (i for i in self)
            out = "myVector2d:({},{})"
        componets = (format(c, fmt_spec) for c in coords)
        return out.format(*componets)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        mev = memoryview(octets[1:]).cast(typecode)
        return cls(*mev)

    def __float__(self):
        return float(abs(self))

    def __int__(self):
        return int(abs(self))

    def __complex__(self):
        return complex(self.x, self.y)

In [2]:
# 测试10 000 000个实例的内存用量

import resource

mem_init = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

vectors = [myVector2d_v02(1.00,3.00) for i in range(10000000)]

mem_final = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

print('Initial RAM usage: {:14,}'.format(mem_init))
print('  Final RAM usage: {:14,}'.format(mem_final))
print('        RAM usage: {:14,}'.format(mem_final-mem_init))

Initial RAM usage:     46,399,488
  Final RAM usage:    628,817,920
        RAM usage:    582,418,432


In [5]:
mem_init = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

vectors = [myVector2d(1.00,3.00) for i in range(10000000)]

mem_final = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

print('Initial RAM usage: {:14,}'.format(mem_init))
print('  Final RAM usage: {:14,}'.format(mem_final))
print('        RAM usage: {:14,}'.format(mem_final-mem_init))

Initial RAM usage:    629,276,672
  Final RAM usage:  2,262,327,296
        RAM usage:  1,633,050,624


## 使用\_\_slots\_\_属性可以减少三分之二的内存消耗！


## 在类中定义 \_\_slots\_\_ 属性之后，实例不能再有\_\_slots\_\_ 中所列名称之外的其他属性。这只是一个副作用，不是 \_\_slots\_\_ 存在的真正原因。不要使用 \_\_slots\_\_ 属性禁止类的用户新增实例属性。\_\_slots\_\_ 是用于优化的，不是为了约 束程序员。

## 然而，“节省的内存也可能被再次吃掉”:
## 如果把 '\_\_dict\_\_' 这个名称添加到 \_\_slots\_\_ 中，实例会在元组中保存各个实例的属性，此外还支持动态创建属性，这些属性存储在常规的 \_\_dict\_\_ 中。当然，把 '\_\_dict\_\_' 添加到 \_\_slots\_\_ 中可能完全违背了初衷，这取决 于各个实例的静态属性和动态属性的数量及其用法。粗心的优化甚至比提早优化还糟糕。

# \_\_slots\_\_ 的问题
总之，如果使用得当，\_\_slots\_\_ 能显著节省内存，不过有几点要注意。
- 每个子类都要定义 \_\_slots\_\_ 属性，因为解释器会忽略继承的 \_\_slots\_\_ 属性。

- 实例只能拥有 \_\_slots\_\_ 中列出的属性，除非把 '\_\_dict\_\_' 加入 \_\_slots\_\_ 中(这样做就去了节省内存的功效)。

- 如果不把 '\_\_weakref\_\_' 加入 \_\_slots\_\_，实例就不能作为弱 引用的目标。

# \_\_slots\_\_ 主要用在固定模式的数据库记录中，用于优化内存。